In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,46169
2,Huelva,Confirmados PDIA 14 días,390
3,Huelva,Tasa PDIA 14 días,"75,99820722177836"
4,Huelva,Confirmados PDIA 7 días,179
5,Huelva,Tasa PDIA 7 dias,"34,88122844281622"
6,Huelva,Total Confirmados,46372
7,Huelva,Curados,45113
8,Huelva,Fallecidos,419


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  46169.0


/tmp/ipykernel_152042/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12683.0


/tmp/ipykernel_152042/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_152042/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_152042/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_152042/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 4110 personas en los últimos 7 días 

Un positivo PCR cada 1798 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,46169.0,179.0,390.0,513170.0,34.881228,75.998207,84.0
Huelva-Costa,27310.0,106.0,229.0,289548.0,36.608783,79.088787,49.0
Condado-Campiña,14343.0,59.0,110.0,156231.0,37.764592,70.408562,32.0
Huelva (capital),12683.0,35.0,80.0,143837.0,24.333099,55.618513,20.0
Cartaya,2124.0,34.0,73.0,20083.0,169.297416,363.491510,13.0
Moguer,2023.0,16.0,30.0,21867.0,73.169616,137.193031,10.0
Aljaraque,1642.0,9.0,10.0,21474.0,41.911148,46.567943,6.0
Palos de la Frontera,1071.0,11.0,21.0,11742.0,93.680804,178.845171,6.0
Isla Cristina,3161.0,8.0,18.0,21393.0,37.395410,84.139672,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,52.0,7.0,30.0,713.0,981.767181,4207.573633,2.0
Granado (El),46.0,3.0,6.0,532.0,563.909774,1127.819549,0.0
Cartaya,2124.0,34.0,73.0,20083.0,169.297416,363.491510,13.0
San Bartolomé de la Torre,385.0,6.0,7.0,3761.0,159.532039,186.120713,3.0
Minas de Riotinto,185.0,4.0,7.0,3812.0,104.931794,183.630640,1.0
Palos de la Frontera,1071.0,11.0,21.0,11742.0,93.680804,178.845171,6.0
Cabezas Rubias,53.0,1.0,1.0,706.0,141.643059,141.643059,0.0
Moguer,2023.0,16.0,30.0,21867.0,73.169616,137.193031,10.0
Santa Bárbara de Casa,47.0,1.0,1.0,1043.0,95.877277,95.877277,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Lepe,3047.0,4.0,18.0,27880.0,14.347202,64.562410,2.0,0.222222
Campofrío,52.0,7.0,30.0,713.0,981.767181,4207.573633,2.0,0.233333
Sierra de Huelva-Andévalo Central,4090.0,14.0,51.0,67391.0,20.774287,75.677761,3.0,0.274510
Nerva,254.0,1.0,3.0,5169.0,19.346102,58.038305,0.0,0.333333
Bollullos Par del Condado,1298.0,2.0,5.0,14387.0,13.901439,34.753597,1.0,0.400000
Palma del Condado (La),1435.0,4.0,10.0,10801.0,37.033608,92.584020,1.0,0.400000
Rociana del Condado,673.0,2.0,5.0,7939.0,25.192090,62.980224,2.0,0.400000
Huelva (capital),12683.0,35.0,80.0,143837.0,24.333099,55.618513,20.0,0.437500
Isla Cristina,3161.0,8.0,18.0,21393.0,37.395410,84.139672,4.0,0.444444
